#  Unit 2.4a Using Programs with Data, SQLAlchemy
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- categories: [t3]

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a defintion of purpose.
    1. Flask app object
    2. SQLAlchemy db object


In [1]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class, purpose and definition.
    - User class: certain information we are collecting about the user
        - a set of developer-defined attributes (characteristics) and methods (behaviors) that you can use to refer to multiple data items as a single entity --> object
    - _init_ method: used to define the attributes/data that we are going to store for each user
    - db.Model inheritance: a parameter
        - you are taking something from db.Model, therefore "inheritance"
        - you take from db.Model so you are able to use the attributes in it
    - ```@property```, ```@<column>.setter```
        - needed for each of the attributes
        - another name for @property is a "getter" --> function called to read the data (R in CRUD) in database
        - @<column>.setter --> function called to create (create = adding user into the object) or update data in database
    - create, read, update, delete methods

In [2]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class CarSpec(db.Model):
    __tablename__ = 'carspecs'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, unique=True, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _type = db.Column(db.String(255), unique=False, nullable=False)
    _seatingCapacity = db.Column(db.String(255), unique=False, nullable=False)
    _powerSource = db.Column(db.String(255), unique=False, nullable=False)
    _transmission = db.Column(db.String(255), unique=False, nullable=False)
    _mileage = db.Column(db.String(255), unique=False, nullable=False)
    _range = db.Column(db.String(255), unique=False, nullable=False)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, type, seatingCapacity, powerSource, transmission, mileage, range):
        self._name = name   # variables with self prefix become part of the object, 
        self._type = type
        self._seatingCapacity = seatingCapacity
        self._powerSource = powerSource
        self._transmission = transmission
        self._mileage = mileage
        self._range = range

    # gets the name of the car
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name

     # gets the type of the car
    @property
    def type(self):
        return self._type
    
    # a setter function, allows type to be updated after initial object creation
    @type.setter
    def type(self, type):
        self._type = type 

    # gets the type of the seating capacity of the car
    @property
    def seatingCapacity(self):
        return self._seatingCapacity
    
    # a setter function, allows seating capacity to be updated after initial object creation
    @seatingCapacity.setter
    def seatingCapacity(self, seatingCapacity):
        self._seatingCapacity = seatingCapacity
    
    # a powerSource getter
    @property
    def powerSource(self):
        return self._powerSource

    # a setter function to set the car's powerSource
    @powerSource.setter
    def powerSource(self, powerSource):
        self._powerSource = powerSource
    
     # a transmission getter
    @property
    def transmission(self):
        return self._transmission

    # a setter function to set the car's transmission
    @transmission.setter
    def transmission(self, transmission):
        self._transmission = transmission
        
     # a mileage getter
    @property
    def mileage(self):
        return self._mileage

    # a setter function to set the car's mileage
    @mileage.setter
    def mileage(self, mileage):
        self._mileage = mileage
    
     # a range getter
    @property
    def range(self):
        return self._range

    # a setter function to set the car's mileage
    @range.setter
    def range(self, range):
        self._range = range
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name" : self.name,
            "type" : self.type,
            "seatingCapacity" : self.seatingCapacity,
            "powerSource" : self.powerSource,
            "transmission" : self.transmission,
            "mileage" : self.mileage,
            "range" : self.range
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", type="", seatingCapacity="", powerSource="", transmission="", mileage="", range=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(type) > 0:
            self.type = type
        if len(seatingCapacity) > 0:
            self.seatingCapacity = seatingCapacity
        if len(powerSource) > 0:
            self.powerSource = powerSource
        if len(transmission) > 0:
            self.transmission = transmission
        if len(mileage) > 0:
            self.mileage = mileage
        if len(range) > 0:
            self.range = range
        # self.price_range = price_range
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object
    2. User Object Constructors
    3. Try / Except 


In [5]:
"""Database Creation and Testing """


# Builds working data for testing
def initCarSpecs():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        s1 = CarSpec(name='Chevrolet Blazer', type='SUV', seatingCapacity='5', powerSource='Gasoline', transmission='Automatic', mileage='24', range='Non-Electric')
        s2 = CarSpec(name='Chevrolet Blazer EV', type='SUV', seatingCapacity='5', powerSource='Electric', transmission='Automatic', mileage='Non-Gasoline', range='400') 
        s3 = CarSpec(name='Chevrolet Bolt EV', type='SUV', seatingCapacity='5', powerSource='Electric', transmission='Automatic', mileage='Non-Gasoline', range='Non-Electric')
        s4 = CarSpec(name='Chevrolet Equinox', type='SUV', seatingCapacity='5', powerSource='Gasoline', transmission='Automatic', mileage='25', range='Non-Gasoline')
        s5 = CarSpec(name='Chevrolet Equinox EV', type='SUV', seatingCapacity='5', powerSource='Electric', transmission='Automatic', mileage='Non-Gasoline', range='300')
        s6 = CarSpec(name='Chevrolet Trailblazer', type='SUV', seatingCapacity='5', powerSource='Gasoline', transmission='Automatic', mileage='17', range='Non-Electric') 
        s7 = CarSpec(name='Chevrolet Traverse', type='SUV', seatingCapacity='7', powerSource='Gasoline', transmission='Automatic', mileage='15', range='Non-Electric')
        s8 = CarSpec(name='Chevrolet Seeker', type='SUV', seatingCapacity='5', powerSource='Gasoline', transmission='Automatic', mileage='30', range='Non-Electric')
        s9 = CarSpec(name='Chevrolet Suburban', type='SUV', seatingCapacity='8', powerSource='Gasoline', transmission='Automatic', mileage='15', range='Non-Electric')
        s10 = CarSpec(name='Chevrolet Tahoe', type='SUV', seatingCapacity='8', powerSource='Gasoline', transmission='Automatic', mileage='17', range='Non-Electric') 
        s11 = CarSpec(name='Chevrolet Colorado', type='Pickup Truck', seatingCapacity='5', powerSource='Gasoline', transmission='Automatic', mileage='16', range='Non-Electric')

    carspecs = [s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11]

    """Builds sample user/note(s) data"""
    for carspec in carspecs:
            try:
                '''add user to table'''
                object = carspec.create()
                print(f"Created new uid {object.name}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist name {carspec.name}, or error.")
                
initCarSpecs()

Records exist name Chevrolet Blazer, or error.
Records exist name Chevrolet Blazer EV, or error.
Records exist name Chevrolet Bolt EV, or error.
Records exist name Chevrolet Equinox, or error.
Records exist name Chevrolet Equinox EV, or error.
Records exist name Chevrolet Trailblazer, or error.
Records exist name Chevrolet Traverse, or error.
Records exist name Chevrolet Seeker, or error.
Records exist name Chevrolet Suburban, or error.
Records exist name Chevrolet Tahoe, or error.
Records exist name Chevrolet Colorado, or error.


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
    1. User.query.filter_by
        - when you do a query.filter_by, you are looking for a user id/checking if it exists
        - `check_credentials` function checks if any existing user IDs are the same one as the current input --> if they're not, then nothing is going to happen
    2. user.password
        - secondary check
        - if there is a user id that is the same as the current input, it does a second check comparing the existing passwords to the password that was inputted
        - if the password is the same, then it returns True. The user already exists

In [4]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        user = cars[CarSpec].query.filter_by(_uid=uid).first() 
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False
        
check_credentials("rey444", "123qwerty")

False

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except
        - allows us to check whether or not a uid with the same name already exists
    2. user = User(...)
        - allows us to initialize the object, which is User
    3. user.dob and try/except
        - if no dob is provided by the user, it sets the user's dob to the current date
    4. user.create() and try/except
        - allows us to know whether or not the object was correctly created

In [24]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(name=name, 
                uid=uid, 
                password=password
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

Created
 {'id': 7, 'name': 'Shreya Sapkal', 'uid': 'rey444', 'dob': '12-21-2004', 'age': 18}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all
        - used to extract all the user data from the database
    2. json_ready assignment, google List Comprehension
        - all users are extracted from databse --> users are turned into JSON formatting
        - why JSON? Because JSON is universal; it is easier to move around the data and use it in other applications.

In [4]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = CarSpec.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[]

# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.

### Adding Update Functionality

In [3]:
import sqlite3

database = 'instance/sqlite.db' # this is location of database

def schema():
    
    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()
    
    # Fetch results of Schema
    results = cursor.execute("PRAGMA table_info('users')").fetchall()

    # Print the results
    for row in results:
        print(row)

    # Close the database connection
    conn.close()
    
schema()

(0, 'id', 'INTEGER', 1, None, 1)
(1, '_name', 'VARCHAR(255)', 1, None, 0)
(2, '_uid', 'VARCHAR(255)', 1, None, 0)
(3, '_password', 'VARCHAR(255)', 1, None, 0)
(4, '_dob', 'DATE', 0, None, 0)


In [4]:
import sqlite3

def update():
    uid = input("Enter user id to update")
    password = input("Enter updated password")
    if len(password) < 2:
        message = "hacked"
        password = 'gothackednewpassword123'
    else:
        message = "successfully updated"

    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    try:
        # Execute an SQL command to update data in a table
        cursor.execute("UPDATE users SET _password = ? WHERE _uid = ?", (password, uid))
        if cursor.rowcount == 0:
            # The uid was not found in the table
            print(f"No uid {uid} was not found in the table")
        else:
            print(f"The row with user id {uid} the password has been {message}")
            conn.commit()
    except sqlite3.Error as error:
        print("Error while executing the UPDATE:", error)
        
    
    # Close the cursor and connection objects
    cursor.close()
    conn.close()
    
update()

The row with user id rey444 the password has been successfully updated


### Adding Delete Functionality

In [5]:
import sqlite3

def delete():
    uid = input("Enter user id to delete")

    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()
    
    try:
        cursor.execute("DELETE FROM users WHERE _uid = ?", (uid,))
        if cursor.rowcount == 0:
            # The uid was not found in the table
            print(f"No uid {uid} was not found in the table")
        else:
            # The uid was found in the table and the row was deleted
            print(f"The row with uid {uid} was successfully deleted")
        conn.commit()
    except sqlite3.Error as error:
        print("Error while executing the DELETE:", error)
        
    # Close the cursor and connection objects
    cursor.close()
    conn.close()
    
delete()

The row with uid whit was successfully deleted
